# CA RNN
Toy example in 1d


In [27]:
'''
Example
'''
import tensorflow as tf
import numpy as np

import utils


class GRUCell(object):
    '''Standard GRU cell.
    
    Calling this cell on a RNN network returns a tuple of activations (h, h)
    '''
    def __init__(self, state_size, activation=tf.sigmoid):
        self._state_size = state_size
        self._activation = activation

    @property
    def state_size(self):
        return self._state_size
    
#     def zero_state(self, batch_size):
#         return 
        
    def __call__(self, x, s, scope=None):
        with scope or tf.name_scope('GRUCell'):
            
            # trainable variables
            with tf.variable_scope('weights', initializer=tf.contrib.layers.xavier_initializer()) as scope:
                W_u = tf.get_variable('W_u', [self._state_size, x[0]])
                W_r = tf.get_variable('W_r', [self._state_size, x[0]])
                W   = tf.get_variable('W', [self._state_size, x[0]])
                U_u = tf.get_variable('U_u', [self._state_size, self._state_size])
                U_r = tf.get_variable('U_r', [self._state_size, self._state_size])
                U   = tf.get_variable('U', [self._state_size, self._state_size])
                b_u = tf.get_variable('b', [self._state_size], initializer=tf.constant_initializer(1.0))
                b_r = tf.get_variable('b', [self._state_size], initializer=tf.constant_initializer(1.0))
                b   = tf.get_variable('b', [self._state_size], initializer=tf.constant_initializer(1.0))
            scope.reuse_variables()
            
            # u = update, r = reset, c = candidates, h = new hidden states
            u = self._activation(tf.matmul(s, U_u) + tf.matmul(x, W_u) + b_u)
            r = self._activation(tf.matmul(s, U_r) + tf.matmul(x, W_u) + b_r)
            c = tf.tanh(tf.matmul(tf.multiply(s, r), U) + tf.multiply(x, W) + b)
            h = tf.multiply(s, u) + tf.multiply((1 - u), c)
            return h, h

        
class ExtendedGRUcell(tf.contrib.rnn.GRUCell):
    def __call__(self, inputs, state, scope=None):
        new_h, new_h = super().__call__(inputs, state, scope=None)
        
        print('extending')
        
        return new_h, new_h
        
        
class ca_rnn(object):
    '''
    A CA RNN neural network tensorflow model.
    
    Aims to generalise a CA algorithm from training data
    
    Properties:
        inference:
        loss:
        optimizer:
        prediction:
    '''
    def __init__(self,
                 x,
                 y=None,
                 learning_rate=1e-4,
                 state_size=5,
                 batch_size=1):
        self._learning_rate = learning_rate
        self._state_size = state_size
        self._batch_size = batch_size
        
        self._create_inference(x)
        self._create_loss(y)
        self._create_optimizer()
#         self._create_prediction(y)
        
    def _create_inference(self, x):
        # create inference
        with tf.name_scope('inference'):
            width = 5
            height = 1
            depth = 1
            classes = 2

            # RNN
            with tf.name_scope('rnn'):
#                 # GRU cell
#                 cell = GRUCell(self._state_size)

#                 '''
#                 c-c-c-c-c
#                 | | | | |
#                 0-0-1-0-1
#                 '''
                
#                 inputs = tf.reshape(x, [self._batch_size, width])

#                 outputs = []
#                 for r, input_ in enumerate(inputs):
#                     call_cell = lambda: cell(input_, state)
                    
#                     (output, state) = call_cell()
#                     outputs.append(output)
                

                # for static_rnn()
#                 reshape = tf.reshape(x, [width, self._batch_size, depth])
#                 inputs = []
#                 inputs = tf.unstack(reshape, axis=0)
                
                # time_major=False
                inputs = tf.reshape(x, [self._batch_size, width, depth])
        
                state_size = 5
                cell = ExtendedGRUcell(state_size)

#                 cell = tf.contrib.rnn.GridLSTMCell(state_size, state_is_tuple=True, num_frequency_blocks=[1])
#                 outputs, final_state = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)
                outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
            
            # Cellular Automata
            outputs = tf.stack(outputs)
            print(outputs.get_shape)
            flatten = tf.reshape(outputs, [self._batch_size, width * state_size])
                

            # Convolution
#             with tf.name_scope('convolution'):
#                 output = tf.reshape(output, [self._batch_size, width, depth])
#                 kernel = tf.Variable(tf.random_normal([3, 1, 1]), dtype=tf.float32, name='kernel')
#                 conv = tf.nn.conv1d(output, kernel, stride=1, padding='SAME')


            # Concatenate
            # and ReLU
            # To softmax classifier
            
        
            # FC1
            num_hidden = 5
            with tf.name_scope('fc1'):
                with tf.variable_scope('output'):
                    W = tf.get_variable('W_out', [num_hidden * width, classes])
                    b = tf.get_variable('b_out', [classes], initializer=tf.constant_initializer(1.0))

#                 flatten = tf.reshape(conv, [self._batch_size, width])
                fc1 = tf.nn.xw_plus_b(flatten, W, b, name='fc1')
#                 print(fc1.get_shape)
                fc1 = tf.reshape(fc1, [self._batch_size, classes])
#                 print(fc1.get_shape)

            self._logits = fc1
        
    def _create_loss(self, y):
        # create loss function
        with tf.name_scope('loss'):
            cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=self._logits, labels=y, name='cross_entropy')
            self._loss = tf.reduce_mean(cross_entropy)
            
    def _create_optimizer(self):
        # create optimizer
            self._optimizer = tf.train.AdamOptimizer(self._learning_rate).minimize(self._loss)
            
#     def _create_prediction(self, y):
        # evaluation
#         with tf.name_scope('prediction'):
#             correct = tf.nn.in_top_k(self._logits, y, 1)
#             self._prediction = tf.reduce_mean(tf.cast(correct, tf.float32))
        
    @property
    def inference(self):
        return self._logits
    
    @property
    def optimizer(self):
        return self._optimizer
    
    @property
    def loss(self):
        return self._loss
    
    @property
    def prediction(self):
        return self._prediction
    

# Reset tensorflow variable names
tf.reset_default_graph()


# Hyperparameters
epochs = 1


# input pipeline
filename_queue = tf.train.string_input_producer(
    ['constrained_100000x5x1x1.tfrecords'], num_epochs=epochs)


# read and decode   
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

feature_map = {
    'x': tf.FixedLenFeature(
        shape=[], dtype=tf.string),
    'y': tf.FixedLenFeature(
        shape=[], dtype=tf.int64,
        default_value=None)
}
parsed = tf.parse_single_example(serialized_example, feature_map)

# decode
width = 5
height = 1
depth = 1

features = tf.decode_raw(parsed['x'], tf.int64)
features = tf.reshape(features, [width, height, depth])
features = tf.cast(features, dtype=tf.float32)
# labels = tf.cast(parsed['y'], dtype=tf.int64)
# labels = tf.squeeze(parsed['y'])
labels = parsed['y']

batch_size = 128
min_after_dequeue = 1000
capacity = min_after_dequeue + 3 + batch_size
example_batch, label_batch = tf.train.shuffle_batch(
    [features, labels], 
    batch_size=batch_size, 
    capacity=capacity,
    allow_smaller_final_batch=True,
    min_after_dequeue=min_after_dequeue)


# construct model
learning_rate = 0.01
model = ca_rnn(
    example_batch, 
    label_batch, 
    batch_size=batch_size, 
    learning_rate=learning_rate)


# train network
init = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()  # for epochs local variable

num_examples = 0
with tf.Session() as sess:
    sess.run([init, init_local])

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#     writer = tf.summary.FileWriter('./graphs/run1', sess.graph)

    try:
        step = 0
        total_loss = 0
        print('# Training')
        while not coord.should_stop():
            step += 1

            loss, yy = sess.run([model.loss, label_batch])
            total_loss += loss

            num_examples = num_examples + batch_size
            if step % 50 == 0:
                print(yy, 'end')
                print('num_examples: {}, average_loss: {}'.format(num_examples, total_loss / step))

            if step == 50000:  # stop early
                coord.request_stop()
            
    except tf.errors.OutOfRangeError:
#         print('Done training for %d epochs, %d steps.' % (num_epochs, step))
        print('Requesting to stop')
        coord.request_stop()
    finally:
        # ask threads to to stop
        coord.request_stop()      

        # wait for threads to finish
        coord.join(threads)
        
        print('# Finished')

#     writer.close()


extending
<bound method Tensor.get_shape of <tf.Tensor 'inference/rnn/rnn/transpose:0' shape=(128, 5, 5) dtype=float32>>
# Training
# Finished


InvalidArgumentError: Received a label value of 4 which is outside the valid range of [0, 2).  Label values: 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
	 [[Node: loss/cross_entropy/cross_entropy = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/cpu:0"](inference/fc1/Reshape, shuffle_batch:1)]]

Caused by op 'loss/cross_entropy/cross_entropy', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/fred/Developer/ca-rnn/venv/lib/python3.6/site-packages/ipykernel/__main__.py", li

ValueError: Shapes (2, 10, 1) and () are incompatible